Experiment A2: Activation Patching with Counterfactuals

Same setup as A1
TODO: record adversarial label to test label correctness

https://nnsight.net/notebooks/tutorials/activation_patching/

In [17]:
import nnsight
from nnsight import NNsight
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from robustness import model_utils
from robustness.datasets import CIFAR

import torchvision.transforms as transforms
from PIL import Image

import os
import sys
# Add the parent directory to the path to import custom modules
sys.path.append(os.path.abspath(os.path.join('..')))
from models.resnet import ResNet18

In [2]:
# load saved model
datapath = '/u/yshi23/distribution-shift/datasets'
model_pth = "/u/yshi23/distribution-shift/adversarial/out/checkpoint.pt.best"
device = "cpu"

# model init using robustness mechanism
# model, _ = model_utils.make_and_restore_model(
#     arch='resnet18', dataset=CIFAR(datapath), resume_path=model_pth, device=device
# )
# Load the checkpoint with map_location to ensure it's on the right device
model = ResNet18(num_classes=10)
checkpoint = torch.load(model_pth, map_location=device)

# Get the state dict
state_dict = checkpoint["model"]

# Create a new state dict with properly formatted keys
new_state_dict = {}
for k, v in state_dict.items():
    # Remove 'module.' prefix if present
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.'
    else:
        name = k
    new_state_dict[name] = v

# Load the modified state dict with strict=False to ignore missing/unexpected keys
model.load_state_dict(new_state_dict, strict=False)

print("Model loaded successfully on", device)

Model loaded successfully on cpu


In [29]:
def load_cifar10_image(image_path):
    # Load the image
    img = Image.open(image_path)
    
    # Define the transforms that match ResNet18 expectations
    # ResNet18 expects 224x224 images, but CIFAR-10 is 32x32
    # We'll resize and normalize according to ImageNet stats as typically used with ResNet
    transform = transforms.Compose([
        transforms.Resize(32),  # Resize to 224x224
        transforms.ToTensor(),   # Convert to tensor
        transforms.Normalize(          # Normalize with CIFAR-10 stats
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2470, 0.2435, 0.2616]
        )
    ])
    
    # Apply transformations
    img_tensor = transform(img)
    
    # Add batch dimension
    img_tensor = img_tensor.unsqueeze(0)  # Shape: [1, 3, 224, 224]
    
    return img_tensor

# Usage example
orig_path = "images/natural_image_0.png"
clean_img = load_cifar10_image(orig_path)

low_ep_path = "images/mask_small_epsilon_0.png"
low_eps_img = load_cifar10_image(low_ep_path)

high_ep_path = "images/mask_large_epsilon_0.png"
high_eps_img = load_cifar10_image(high_ep_path)


In [7]:
# Get all layer names (keys) from the model's state dictionary
layer_names = list(model.state_dict().keys())

# Print all layer names
print("Model layer names:")
for name in layer_names:
    print(name)

Model layer names:
conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
bn1.num_batches_tracked
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.bn2.num_batches_tracked
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.bn1.running_mean
layer2.0.bn1.running_var
layer2.0.bn1.num_batches_tracked
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.bn2.running_mean
layer2.0.bn2.running_var
layer2.0.bn2.num_batches_tracked
layer2.0.shortcut.0.weight


In [ ]:

with nnsight_model.trace(low_eps_img) as tracer:
    # Forward pass through the model
    # low eps adversarial

    conv1_out = nnsight_model.conv1.output.save()
    layer1_out = nnsight_model.layer1.output.save()
    layer2_out = nnsight_model.layer2.output.save()
    layer3_out = nnsight_model.layer3.output.save()
    layer4_out = nnsight_model.layer4.output.save()
    linear_out = nnsight_model.linear.output.save()

# edit the og model first with conv1_out
with nnsight_model.edit() as low_eps_conv1_edit:
    nnsight_model.layer4.output = layer4_out

# now get the prediction confidence of the edited model 
print("edited: ", low_eps_conv1_edit(clean_img))
print("original: ", nnsight_model(clean_img))
print("original with adv input: ", nnsight_model(low_eps_img))

edited:  tensor([[ 0.4219,  0.2396,  0.4102,  0.8014,  0.3155,  0.1142,  0.5998, -0.8716,
          0.9336,  0.4383]], grad_fn=<AddmmBackward0>)
original:  tensor([[ 0.4219,  0.2396,  0.4102,  0.8014,  0.3155,  0.1142,  0.5998, -0.8716,
          0.9336,  0.4383]], grad_fn=<AddmmBackward0>)


In [ ]:
orig_class_idx = 0
small_eps_class_idx = 4
large_eps_class_idx = 6


Available top-level attributes: ['R', 'add', 'apply', 'args', 'backend', 'cond', 'execute', 'graph', 'invoke', 'invoker', 'is_protocol', 'iter', 'kwargs', 'local', 'log', 'stop', 'style', 'vis']
Error accessing sublayers: 'InterleavingTracer' object has no attribute 'layer1'
